# Download and preprocess data

This notebook is intended to give an intro into how to download and preprocess climate data for climate model debiasing and further use. There are many ways to access climate data -- often on different temporal or spatial resolution (that might still be supported by PACKATE_NAME). This notebook is meant to illustrate one possible way.

To apply debiasing three datasets are needed: observations, often from a reanalasysis product but also other datasets are possible, to provide a baseline; historical climate model data in a reference period to compare with observations; and future climate model data that is debiased. Here we will use **CMIP6 data** from the Climate Data Store (CDS) as climate model input and download and preprocess two reanalysis as observations: 1) **ERA5** from the CDS and 2) **NCEP/DOE Reanalysis II** from the PSL datastore (NOAA).

We work on daily temporal resolution here. Unfortunately not all CMIP6 models in the CDS are currently available on such resolution, so we need to select what covariates to use. Climate model processing and debiasing is of course also possible on monthly resolution: the choice here is mainly illustrative.

In [1]:
import os
import urllib

# Scientific computing
import iris
import xarray
import numpy

## 1. Download data

Let's fix a data-directory where our inputs will go and create it if it doesn´t exist:

In [2]:
DATADIR = "data_download_and_preprocessing"

if not os.path.exists(DATADIR):
    os.mkdir(DATADIR)

We will first download climate model data from the CDS. We will use CMIP6 data on daily resolution.

### 1.1. Download climate model data

To request climate data from the Climate Data Store (CDS) we will use the CDS API. Let's install it if it isn't installed:

In [3]:
#!pip install cdsapi
import cdsapi

We make use of the option to manually set the CDS API credentials. First, you have to define two variables: URL and KEY which build together your CDS API key. The string of characters that make up your KEY include your personal User ID and CDS API key. To obtain these, first register or login to the CDS (http://cds.climate.copernicus.eu), then visit https://cds.climate.copernicus.eu/api-how-to and copy the string of characters listed after "key:". Replace the ######### below with this string

In [4]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '140375:f24956b6-43ca-40ed-9563-be7031a4b2c3' # enter your key instead

Let's choose a model and variable we are interested in, as well as fix some meta-paramters. If we are interested in multiple variable we can just iterate the code below:

In [5]:
# choose model
model = 'mpi_esm1_2_lr'

# choose variables to extract (not all variables available at daily resolution for all cmip6 models at the moment)
variable = 'precipitation'

# choose area to extract
area = [80, 3, 20, 30]

# choose a historical period to extract
period_hist = '1979-01-01/2015-12-31'

# choose a future period to extract:
period_fut = '2050-01-01/2070-12-31'

# choose a filename for the historical cm data
fname_cm_hist = f"cmip6_daily_1979-2015_ipsl_historical_{variable}.zip"

# choose a filename for the future cm data
fname_cm_future = f"cmip6_daily_2050-2070_ipsl_ssp5_8_5_{variable}.zip"



Both datasets will be in `DATADIR` under `fname_cm_hist` and `fname_cm_future`.

#### 1.1.1. Download historical climate model data

First we will download the historical climate model data:

In [6]:
# download historical climate model data

c = cdsapi.Client(url=URL, key=KEY)

c.retrieve(
    'projections-cmip6',
    {
        'temporal_resolution': 'daily',
        'experiment': 'historical',
        'level': 'single_levels',
        'variable': variable,
        'model': model,
        'date': period_hist,
        'area': area,
        'format': 'zip',
    },
    f'{DATADIR}/{fname_cm_hist}')

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:04:52,277 INFO Welcome to the CDS
2022-08-22 13:04:52,278 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:04:52,852 INFO Request is completed
2022-08-22 13:04:52,854 INFO Downloading https://download-0000-clone.copernicus-climate.eu/ca

Result(content_length=16125737,content_type=application/zip,location=https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data9/adaptor.esgf_wps.retrieve-1661033719.217655-601-14-369cd835-15d6-4a0d-880c-000dfe2e31bf.zip)

We need to unzip the folder:

In [263]:
import zipfile

with zipfile.ZipFile(f'{DATADIR}/{fname_cm_hist}', 'r') as zip_ref:
    zip_ref.extractall(DATADIR)

The file is in `DATADIR/pr_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_19790101-20141231_v2019071.nc`

#### 1.1.2. Download future climate model data

And now the future climate model data:

In [7]:
# download future climate model data

c = cdsapi.Client(url=URL, key=KEY)

c.retrieve(
    'projections-cmip6',
    {
        'temporal_resolution': 'daily',
        'experiment': 'ssp5_8_5',
        'level': 'single_levels',
        'variable': variable,
        'model': model,
        'date': period_fut,
        'area': area,
        'format': 'zip',
    },
    f'{DATADIR}/{fname_cm_future}')

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:04:59,648 INFO Welcome to the CDS
2022-08-22 13:04:59,650 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:04:59,766 INFO Request is completed
2022-08-22 13:04:59,769 INFO Downloading https://download-0004-clone.copernicus-climate.eu/ca

Result(content_length=8643246,content_type=application/zip,location=https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data6/adaptor.esgf_wps.retrieve-1661033873.2348728-23827-20-937b4c7b-1630-4e08-9600-6d3d1912cc8b.zip)

We need to unzip the folder:

In [264]:
import zipfile

with zipfile.ZipFile(f'{DATADIR}/{fname_cm_future}', 'r') as zip_ref:
    zip_ref.extractall(DATADIR)

The file is in `DATADIR/pr_day_MPI-ESM1-2-LR_ssp585_r1i1p1f1_gn_20500101-20701231_v20190710.nc`

### 1.2. Download observations

Now we will download observations. We will first download ERA5 data from the CDS and afterwards the NCEP/DOE II Reanalysis from the PSL.

#### 1.2.1. Download ERA5

We will download ERA5 on daily temporal resolution (if the climate model were on other temporal resolutions we would also need a different one for ERA5). The script is inspired by [this discussion](https://confluence.ecmwf.int/pages/viewpage.action?pageId=228867588) and uses the [
Daily statistics calculated from ERA5 data
](https://cds.climate.copernicus.eu/cdsapp#!/software/app-c3s-daily-era5-statistics?tab=overview) application. The output of this application is a separate netCDF file for chosen daily statistic for each month for each year. We concatenate those files then manually. First we need to make some selections (make sure the data chosen here is consistent with the cm data pulled above):

In [8]:
# choose years to request (this should overlap with the `period_hist` chosen for the cm data)*
years =  map(str, range(1979, 2016))

# choose months to request
months = map(str, range(1, 13))

# choose a variable (must be a valid ERA5 CDS API name)
variable = "total_precipitation"

# choose a required statistic (valid names given in the application description above)
statistic = "daily_mean"

# choose an area (should be the same as above)
area = {"lat": [30, 80], "lon": [3, 20]}

# choose a filename
fname_era5 = f"era5_{variable}_{statistic}_1979_2016.nc"

And now we can request the data:

In [10]:
c = cdsapi.Client(url=URL, key=KEY, timeout=300)

# Loop over years and months
filenames_for_cleanup= []
for yr in years:
    for mn in months:
        result = c.service(
            "tool.toolbox.orchestrator.workflow",
            params={
                 "realm": "user-apps",
                 "project": "app-c3s-daily-era5-statistics",
                 "version": "master",
                 "kwargs": {
                     "dataset": "reanalysis-era5-single-levels",
                     "product_type": "reanalysis",
                     "variable": variable,
                     "statistic": statistic,
                     "year": yr,
                     "month": mn,
                     "time_zone": "UTC+00:0",
                     "frequency": "1-hourly",
                     "grid": "1.0/1.0",
                     "area": area,
                     },
            "workflow_name": "application"
        })  

 
        filename = f"{DATADIR}/era5_{variable}_{statistic}_{yr}_{mn}.nc"
        url = result[0]['location']

        # Download nc file
        urllib.request.urlretrieve(url, filename)
        # Append filename to list of filenames to cleanup
        filenames_for_cleanup.append(filename)

# Combine monthly data
combined_data = xarray.open_mfdataset(f"{DATADIR}/era5_{variable}_{statistic}_*.nc", combine = 'nested', concat_dim="time")
combined_data.to_netcdf(f"{DATADIR}/{fname_era5}")

# Cleanup
for filename in filenames_for_cleanup:
    os.remove(filename)

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:06:00,526 INFO Welcome to the CDS
2022-08-22 13:06:00,527 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-29755ae982cf4e2da9b6f738befb90f6
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-08-22 13:06:00,619 INFO Request is queued
/home/jakobwes/anaconda3/envs/ib

#### 1.2.2. Download NCEP/DOE II

After downloading the ERA5-data we can now download the [NCEP/DOE II data](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis2.html). [Here is an overview](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis2.html) of the possible variables and we take the data from [the datastore here](https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/Dailies/gaussian_grid/). First some selections again:

In [83]:
# Variable name. Needs to be one of the NCEP-names in https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/Dailies/gaussian_grid/. 
variable = "prate.sfc.gauss"

# choose years to request (this should overlap with the `period_hist` chosen for the cm data)*
years =  ["2009"]#map(str, range(1979, 2016))

# choose an area (should be the same as above)
area = {"lat": [30, 80], "lon": [3, 20]}

# choose a filename
fname_ncep_doe = f"ncep_doe_{variable}_1979_2016.nc"

Now we can download the data:

In [259]:
# Download data year by year
filenames_for_cleanup = []
for year in years:
    url = f"https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/Dailies/gaussian_grid/{variable}.{year}.nc"
    filename = f"{DATADIR}/{variable}_{year}.nc"
    # Download nc file
    urllib.request.urlretrieve(url, filename)
    # Append filename to list of filenames to cleanup
    filenames_for_cleanup.append(filename)

# Combine data for variable
combined_data = xarray.open_mfdataset(f"{DATADIR}/{variable}_*.nc", combine = 'nested', concat_dim="time")
# Select area
combined_data = combined_data.sel(lon=slice(min(area["lon"]), max(area["lon"])),lat=slice(max(area["lat"]), min(area["lat"])))
# Write to file                                                                                        
combined_data.to_netcdf(f"{DATADIR}/{fname_ncep_doe}")
# Cleanup
for filename in filenames_for_cleanup:
    os.remove(filename)


## 2. Convert and regrid data

Now that we have downloaded the data we need to make sure:

1) Observations and cm data are on the same temporal resolution: this is covered because we downloaded the data on daily resolution.
2) Observations and cm data are on the same spatial resolution: we need to regrid the data.
3) Observations and cm data are on the same units: we may need to convert units.

Furthermore we might want to extract additional information and need to get the numpy arrays corresponding to the data. In the numpy arrays we need to make sure that they have the form `[t,x,y]`.


### 2.1. Regrid data 

Now that we have data on the same temporal resolution for both the climate model and observations we need to make sure they are also on the same spatial one and regrid the datasets. The climate model data is on a coarser grid, therefore we will regrid the observational data onto this resolution. However there are also other solutions, where the [climate model data is regridded onto the resolution of the observations](https://esd.copernicus.org/articles/9/313/2018/).

We will use iris for the regridding, however there are also xarray solutions. Different variables might require different regridding schemes: [a list of ones available in iris is given here](https://scitools-iris.readthedocs.io/en/latest/userguide/interpolation_and_regridding.html?highlight=interpolate#regridding). For precipitation we choose a regridder based on Nearest values. Other regridders like linear ones *might* introduce negative values.

In [278]:
cm_hist = iris.load_cube(f"{DATADIR}/pr_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_19790101-20141231_v20190710.nc", "precipitation_flux")
cm_future = iris.load_cube(f"{DATADIR}/pr_day_MPI-ESM1-2-LR_ssp585_r1i1p1f1_gn_20500101-20701231_v20190710.nc", "precipitation_flux")

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'pr'
  warnings.warn(
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/cf.py:1154: UserWarning: Ignoring variable 'lat_bnds' referenced by variable 'lat': Dimensions ('time', 'lat', 'bnds') do not span ('lat',)
  warnings.warn(msg)
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/cf.py:1154: UserWarning: Ignoring variable 'lon_bnds' referenced by variable 'lon': Dimensions ('time', 'lon', 'bnds') do not span ('lon',)
  warnings.warn(msg)
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'pr'
  warnings.warn(
/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/cf.py:1154: UserWarn

First let's take care of the ERA5 reanalysis:

In [325]:
obs_era5 = iris.load_cube(f"{DATADIR}/{fname_era5}")
obs_era5 = obs_era5.regrid(cm_hist, iris.analysis.Nearest())

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/_nc_load_rules/helpers.py:874: UserWarning: Failed to create 'time' dimension coordinate: The 'time' DimCoord points array must be strictly monotonic.
Gracefully creating 'time' auxiliary coordinate instead.
  warnings.warn(msg.format(name=str(cf_coord_var.cf_name), error=e_msg))


And now of the NCEP/DOE II one:

In [285]:
obs_ncep_doe = iris.load_cube(f"{DATADIR}/{fname_ncep_doe}")
obs_ncep_doe = obs_ncep_doe.regrid(cm_hist, iris.analysis.Nearest())

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/iris/fileformats/_nc_load_rules/helpers.py:645: UserWarning: Ignoring netCDF variable 'prate' invalid units 'Kg/m^2/s'
  warnings.warn(msg)


### 2.1. Extract additional information

The data objects are now all on the same temporal and spatial resolution. Sometimes it is useful to extract the dates:

In [317]:
def get_dates(x):
    time_dimension = x.coords()[0]
    dates = time_dimension.units.num2date(time_dimension.points)
    return dates

get_dates = np.vectorize(get_dates)

dates_cm_hist = get_dates(cm_hist)
dates_cm_future = get_dates(cm_future)

dates_obs_era5 = get_dates(obs_era5)
dates_obs_ncep_doe = get_dates(obs_ncep_doe)

DimCoord(array([ 3.75 ,  5.625,  7.5  ,  9.375, 11.25 , 13.125, 15.   , 16.875,
       18.75 , 20.625, 22.5  , 24.375, 26.25 , 28.125, 30.   ]), standard_name='longitude', units=Unit('degrees'), long_name='Longitude', var_name='lon')

### 2.3. Get numpy arrays

To work with ibias (XXXX) we need to get the numpy arrays associated with the data:

In [332]:
cm_hist = cm_hist.data
cm_future = cm_future.data

obs_era5 = obs_era5.data
obs_ncep_doe = obs_ncep_doe.data

### 2.4. Convert units

From the [ERA5 documentation](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation) we can see that the precipitation is measured in m, whilst in `cm_hist` and `cm_future` it is measured as flux (m / s^-1). To convert we need to divide the ERA5-values by 86 400 (the number of seconds per day):

In [333]:
obs_era5 = obs_era5/86400

The values in the NCEP/DOE II reanalysis are in the same units (check! XXXX)

## 3. Apply debiaser

After these preparations we can finally apply a debiaser. For example we can use quantile mapping. For the ERA5 data:

In [ ]:
from PACKAGE_NAME.debias import QuantileMapping

debiaser = QuantileMapping.from_variable("pr")

In [ ]:
debiased_cm_future_era5 = debiaser.apply(obs_era5, cm_hist, cm_future)

and for the NCEP/DOE II data:

In [ ]:
debiased_cm_future_ncep_doe = debiaser.apply(obs_ncep_doe, cm_hist, cm_future)